In [1]:
from legal_backend.r_to_py.r_to_py import init_R
from legal_backend.bartCause.bart_cause import BARTCause
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
init_R()

### Infer Law Effect

In [3]:
#load csv files and load possible laws
laws_df = pd.read_csv('laws.csv')
opi_df = pd.read_csv('opioid2013.csv')

law_names = list(laws_df.columns)[1:]

In [6]:
opi_df

,fips,state_name,county_name,percent_of_population_under_15_(2013_2017),percent_of_population_aged_15_to_64_(2013_2017),percent_of_population_65plus_(2013_2017),percent_of_population_white_(non_hispanic)_(2013_2017),percent_of_populaiton_black_(non_hispanic)_(2013_2017),percent_of_population_hispanic_or_latino_(2013_2017),percent_of_population_asian_(non_hispanic)_(2013_2017),...,number_of_mental_health_facilities_(as_of_march_2023),mdspercapita,urbanicity,social_resilience_score,economic_risk_score,social_risk_score,economic_resilience_score,prosperity_index_score,death_rate_2013_2016,death_rate
0,1001,Alabama,Autauga County,0.2040,0.6562,0.1398,0.7568,0.1837,0.0257,0.0081,...,0.0,3.63,1.0,4.0,2.0,2.0,5.0,3.0,16.349071,-0.895509
1,1003,Alabama,Baldwin County,0.1830,0.6298,0.1871,0.8318,0.0923,0.0437,0.0066,...,1.0,7.52,1.0,3.0,1.0,2.0,2.0,2.0,34.584733,21.770410
2,1005,Alabama,Barbour County,0.1794,0.6553,0.1653,0.4589,0.4789,0.0431,0.0044,...,1.0,0.00,0.0,5.0,5.0,3.0,4.0,5.0,26.301946,9.328918
3,1007,Alabama,Bibb County,0.1702,0.6809,0.1489,0.7477,0.2121,0.0222,0.0006,...,0.0,8.86,1.0,4.0,4.0,4.0,3.0,4.0,26.581606,18.037349
4,1009,Alabama,Blount County,0.1942,0.6339,0.1719,0.8766,0.0156,0.0873,0.0016,...,1.0,0.00,1.0,5.0,2.0,3.0,5.0,4.0,64.120338,-25.060492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132,56037,Wyoming,Sweetwater County,0.2335,0.6723,0.0942,0.7982,0.0087,0.1586,0.0062,...,2.0,8.93,0.0,4.0,3.0,2.0,3.0,3.0,53.557083,10.033675
3133,56039,Wyoming,Teton County,0.1673,0.7143,0.1184,0.8120,0.0061,0.1517,0.0181,...,2.0,13.26,0.0,1.0,3.0,1.0,1.0,1.0,22.101401,7.917039
3134,56041,Wyoming,Uinta County,0.2493,0.6439,0.1068,0.8772,0.0019,0.0896,0.0015,...,4.0,14.36,0.0,4.0,2.0,1.0,2.0,2.0,52.649213,-18.526175
3135,56043,Wyoming,Washakie County,0.1997,0.6038,0.1965,0.8240,0.0079,0.1396,0.0018,...,1.0,11.97,0.0,3.0,1.0,3.0,2.0,2.0,23.949228,14.670752


In [4]:
# pick a law of interest
law = law_names[1]
print(law)

ad_providers_Physicians


In [5]:
new_cols = list(opi_sm_df.columns[:28])
new_cols.append(law)

In [6]:
df= opi_sm_df[new_cols]
df.head()

,percent_of_population_under_15_(2013_2017),percent_of_population_aged_15_to_64_(2013_2017),percent_of_population_65plus_(2013_2017),percent_of_population_white_(non_hispanic)_(2013_2017),percent_of_populaiton_black_(non_hispanic)_(2013_2017),percent_of_population_hispanic_or_latino_(2013_2017),percent_of_population_asian_(non_hispanic)_(2013_2017),percent_of_population_native_american_alaska_native_(non_hispanic)_(2013_2017),percent_of_population_native_hawaiian_pacific_islander_(2013_2017),percent_of_population_aged_25plus_who_have_at_least_a_high_school_diploma_(2013_2017),...,number_of_mental_health_facilities_(as_of_march_2023),mdspercapita,urbanicity,social_resilience_score,economic_risk_score,social_risk_score,economic_resilience_score,prosperity_index_score,delta_death_rate,ad_providers_Physicians
0,0.1028,0.8306,0.0667,0.1491,0.0661,0.1003,0.2946,0.3331,0.0109,0.8436,...,2,0.00,0,5,3,2,5,4,-0.978100,0
1,0.1847,0.7357,0.0796,0.5308,0.0032,0.0488,0.0117,0.3174,0.0021,0.9260,...,0,0.00,0,3,2,3,2,2,0.000000,0
2,0.1525,0.7746,0.0729,0.8527,0.0099,0.0080,0.0339,0.0311,0.0000,0.9621,...,2,0.00,0,1,4,2,3,2,-1.334086,0
3,0.1523,0.6820,0.1657,0.7690,0.0004,0.0162,0.0336,0.0475,0.0020,0.9545,...,1,39.56,0,1,3,1,1,1,8.107480,0
4,0.1623,0.6711,0.1666,0.4832,0.0346,0.0498,0.0228,0.3080,0.0005,0.9056,...,0,0.00,0,2,4,5,2,3,-4.487059,0


In [7]:
# construct train test data 
train_idxs, test_idxs = train_test_split(list(df.index), test_size=0.3, random_state=99)


X_df = df.iloc[:, :27].to_numpy()
M = X_df.shape[1] 

y = df[['delta_death_rate']].to_numpy()
Z_law = df[[law]].to_numpy()
# numerical columns
num_cols = [c for c in range(M) if len(np.unique(X_df[:, c])) > 2] 


X_train = X_df[train_idxs,:]
X_test = X_df[test_idxs,:]

y_train = y[train_idxs,:]
y_test= y[test_idxs,:]

Z_train = Z_law[train_idxs,:]
Z_test= Z_law[test_idxs,:]

# standardize data
scaler_ = preprocessing.StandardScaler().fit(X_train[:,num_cols])
X_train_scaled = np.copy(X_train)
X_train_scaled[:,num_cols] = scaler_.transform(X_train[:,num_cols])

X_test_scaled = np.copy(X_test)
X_test_scaled[:,num_cols] = scaler_.transform(X_test[:,num_cols])


In [8]:
bart_eval = BARTCause()

In [9]:
bart_eval.fit(X_train_scaled, y_train, Z_train, n_samples=1000,  n_burn=0,  n_chains=5)

In [10]:
def rmse(y, y_pred):
    rmse = np.sqrt((np.sum((y - y_pred) ** 2) / y.shape[0]))
    return rmse

In [11]:
# evaluate BART fit on response surface
newData = np.concatenate((X_test_scaled, Z_test), axis=1)

y_test_predicted_ = bart_eval.predict(newData, infer_type="mu")
y_test_predicted = y_test_predicted_[:,np.newaxis]

In [12]:
print("BART RMS:", rmse(y_test,y_test_predicted),"\n", "Baseline RMS:", rmse(y_test,y_test.mean()))

BART RMS: 16.63682298548655 
 Baseline RMS: 15.418032984620135


In [ ]:
def r_square(y, y_pred):
    # residual sum of squares
    ss_res = np.sum((y - y_pred) ** 2)

    # total sum of squares
    ss_tot = np.sum((y - np.mean(y)) ** 2)

    # r-squared
    r2 = 1 - (ss_res / ss_tot)
    return r2

In [ ]:
# evaluate BART fit on response surface
newData = np.concatenate((X_test_scaled, Z_test), axis=1)

y_test_predicted_ = bart_eval.predict(newData, infer_type="mu")
y_test_predicted = y_test_predicted_[:,np.newaxis]

r_square(y_test,y_test_predicted)

0.09132420219802528